## Demonstration: CRC model
The following code is a small example from the original DL module for proteomic analysis of colorectal tissue samples. 


--> The .mzmL or .d (bruker) files produced from a mass spectrometry experiment were further analysed using DIA-NN software. The output was .csv files which contained information about predicted parameters like precursor intensity, precurson quantity, Retention time etc. 

--> While in the original project the aim was to compare the protein expression including peptide analysis; between tumor and normal tissue dataset, in this project, I have only compared the normalised peptide intensities.

--> I have only one (sample) file for each category, which is reflected in model performance.  




### 1. Import libraries

In [43]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from keras_tuner import HyperModel, Hyperband

### 2. Load and Pre-process the data 

In [ ]:
def load_and_preprocess_data():
    tumor_data = pd.read_csv('tumor.csv')
    normal_data = pd.read_csv('normal.csv')

    columns_of_interest = [
        'Protein.Names',
        'Modified.Sequence',  # Peptide sequences
        'Precursor.Quantity', # Peptide intensities
        'Precursor.Charge',
        'RT',
    ]

## Clean Protein--->[sometimes protein names come in semi-colon seperated format, so taking first protein name to standardise]

    def clean_protein_names(protein_name):
        return protein_name.split(';')[0] if pd.notnull(protein_name) else protein_name
    
## Normalised Peptide Intensities--->[each protein has several peptides, their intensity is normalised so that there is no bias from number of peptides per protein]

    def normalize_intensity(data):
        return data.groupby('Protein.Names')['Precursor.Quantity'].apply(lambda x: x / x.sum()).reset_index(drop=True)

    def filter_and_normalize(data):
        filtered_data = data[columns_of_interest].copy()
        filtered_data['Protein.Names'] = filtered_data['Protein.Names'].apply(clean_protein_names)
        filtered_data['Normalized.Intensity'] = normalize_intensity(filtered_data)
        return filtered_data

    filtered_tumor_data = filter_and_normalize(tumor_data)
    filtered_normal_data = filter_and_normalize(normal_data)

    filtered_tumor_data.to_csv("filtered_data_tumor.csv", index=False)
    filtered_normal_data.to_csv("filtered_data_normal.csv", index=False)

    print('Filtered tumor data saved to: filtered_data_tumor.csv')
    print('Filtered normal data saved to: filtered_data_normal.csv')
    
    return filtered_tumor_data, filtered_normal_data

### 3. Prepare the features and labels for modeling

In [ ]:

def prepare_features_and_labels(tumor_data, normal_data):
    tumor_data['label'] = 1
    normal_data['label'] = 0

    data = pd.concat([tumor_data, normal_data], ignore_index=True)

    X_numeric = data[['Normalized.Intensity', 'RT']].values
    y = data['label'].values

    one_hot_encoder = OneHotEncoder(sparse_output=False)
    categorical_features = one_hot_encoder.fit_transform(data[['Precursor.Charge']])

    X = np.concatenate([X_numeric, categorical_features], axis=1)

    return X, y




### 4. Model building using Keras Tuner for hyperparameter optimization

In [ ]:
def build_model(hp):
    model = Sequential([
        Input(shape=(X_train.shape[1],)),
        Dense(hp.Int('units_layer1', min_value=32, max_value=128, step=32), activation='relu'),
        Dropout(hp.Float('dropout_layer1', 0.2, 0.5, step=0.1)),
        Dense(hp.Int('units_layer2', min_value=16, max_value=64, step=16), activation='relu'),
        Dropout(hp.Float('dropout_layer2', 0.2, 0.5, step=0.1)),
        Dense(1, activation='sigmoid')
    ])

    model.compile(
        optimizer=Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    
    return model

### 5. Train and evaluate the model

In [ ]:
def train_and_evaluate(X_train, X_test, y_train, y_test):
    tuner = Hyperband(
        build_model,
        objective='val_accuracy',
        max_epochs=10,
        factor=3,
        directory='my_dir',
        project_name='hyperparameter_tuning'
    )

    early_stopping = EarlyStopping(monitor='val_loss', patience=3)

    tuner.search(
        X_train, y_train,
        validation_data=(X_test, y_test),
        epochs=50,
        callbacks=[early_stopping],
        verbose=2
    )

    best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
    print(f"Best units_layer1: {best_hps.get('units_layer1')}")
    print(f"Best dropout_layer1: {best_hps.get('dropout_layer1')}")
    print(f"Best units_layer2: {best_hps.get('units_layer2')}")
    print(f"Best dropout_layer2: {best_hps.get('dropout_layer2')}")
    print(f"Best learning rate: {best_hps.get('learning_rate')}")

    best_model = tuner.hypermodel.build(best_hps)

    loss, accuracy = best_model.evaluate(X_test, y_test)
    print(f'Test Accuracy: {accuracy:.4f}')

    best_model.save('best_model.keras')

In [54]:
if __name__ == "__main__":
    tumor_data, normal_data = load_and_preprocess_data()
    X, y = prepare_features_and_labels(tumor_data, normal_data)

    # Split the dataset into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Standardize the features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Train and evaluate the model
    train_and_evaluate(X_train_scaled, X_test_scaled, y_train, y_test)

Filtered tumor data saved to: filtered_data_tumor.csv
Filtered normal data saved to: filtered_data_normal.csv
Reloading Tuner from my_dir\hyperparameter_tuning\tuner0.json
Best units_layer1: 32
Best dropout_layer1: 0.30000000000000004
Best units_layer2: 16
Best dropout_layer2: 0.2
Best learning rate: 0.01
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.3033 - loss: 0.8139  
Test Accuracy: 0.3193


### 6. Perform K-Fold cross-validation

In [ ]:

def perform_cross_validation(X, y, n_splits=5):
    # Standardize features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # Initialize KFold
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    val_scores = []

    # Early stopping callback
    early_stopping = EarlyStopping(monitor='val_loss', patience=3, verbose=0)

    for fold, (train_index, val_index) in enumerate(kf.split(X_scaled), 1):
        X_train_fold, X_val_fold = X_scaled[train_index], X_scaled[val_index]
        y_train_fold, y_val_fold = y[train_index], y[val_index]

        # Model architecture
        fold_model = Sequential([
            Dense(64, input_shape=(X_train_fold.shape[1],), activation='relu'),
            Dropout(0.5),
            Dense(32, activation='relu'),
            Dropout(0.5),
            Dense(1, activation='sigmoid')
        ])

        # Compile the model
        fold_model.compile(optimizer='adam',
                           loss='binary_crossentropy',
                           metrics=['accuracy'])

        # Train the model
        fold_model.fit(X_train_fold, y_train_fold,
                       validation_data=(X_val_fold, y_val_fold),
                       epochs=50,
                       batch_size=32,
                       callbacks=[early_stopping],
                       verbose=0)  # Use verbose=0 to hide training output during CV

        # Evaluate the model on the validation set
        val_loss, val_accuracy = fold_model.evaluate(X_val_fold, y_val_fold, verbose=0)
        print(f'Validation Accuracy for Fold {fold}: {val_accuracy:.4f}')
        val_scores.append(val_accuracy)

    # Calculate mean accuracy across all folds
    mean_val_accuracy = np.mean(val_scores)
    print(f'Mean Validation Accuracy: {mean_val_accuracy:.4f}')

    
if __name__ == "__main__":
    perform_cross_validation(X, y)

c:\Users\sharm\Desktop\UNI\bioinformatics\.venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Validation Accuracy for Fold 1: 0.7048
Validation Accuracy for Fold 2: 0.6687
Validation Accuracy for Fold 3: 0.7590
Validation Accuracy for Fold 4: 0.6687
Validation Accuracy for Fold 5: 0.6848
Mean Validation Accuracy: 0.6972
